### Greedy algorithm for reducing redundant moves

In [6]:
import json
import copy
from collections import Counter
from functools import partial

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# load sample solutions
sample_submission_arr = pd.read_csv("./puzzles/sample_submission.csv").to_numpy()
# load all the puzzle's start and end positions
puzzles_arr = pd.read_csv("./puzzles/puzzles.csv").to_numpy()

puzzle_solutions = {}

for puzzle, solution in zip(puzzles_arr, sample_submission_arr):
    puzzle_name = puzzle[1]
    idx = puzzle[0]
    steps = solution[1].split(".")
    puzzle_solutions[(puzzle_name, idx)] = steps

print(list(puzzle_solutions.items())[0])

(('cube_2/2/2', 0), ['r1', '-f1'])


In [28]:
# some low hanging fruits are:
# cancel pairs of inverses, e.g. (f0, -f0)
# cancel 4 rotations
# change 3 rotations to 1 rotation e.g. (f0, f0, f0 == -f0)


def remove_prefix(text, prefix):
    return text[text.startswith(prefix) and len(prefix) :]


def get_inverse_move(move_name):
    if move_name.startswith("-"):
        return move_name[1:]
    else:
        return f"-{move_name}"


def reduce_wreath(subsequence):
    """For wreath we can only cancel consecutive, opposite pairs"""
    # we use a simple stack to do this, we should only need one pass for this
    out = []
    prev = None
    for move in subsequence:
        prev = out[-1] if out else None
        if prev and get_inverse_move(prev) == move:
            out.pop()
        else:
            out.append(move)

    return out


def reduce_globe(subsequence, n):
    if not subsequence:
        return []
    move_type = remove_prefix(subsequence[0], "-")[0]
    if move_type == "r":
        out = reduce_commutative(subsequence, n)
    elif move_type == "f":
        out = []
        prev = None
        for move in subsequence:
            prev = out[-1] if out else None
            if prev and remove_prefix(prev, "-") == move:
                out.pop()
            else:
                out.append(move)

    return out


def reduce_commutative(subsequence, n):
    """Given a subsequence of parallel moves, return the reduced moves"""
    subsequence.sort()

    out = []
    curr_count, curr_move = 0, ""
    # reduce 3 or 4 in a row:
    for move in subsequence + ["x"]:
        # a dummy item 'x' to make loop run once more for leftover items
        if move != curr_move:
            if curr_count == n - 1:
                out.append(get_inverse_move(curr_move))
            else:
                out.extend([curr_move] * curr_count)
            curr_count, curr_move = 1, move
        elif move == curr_move:
            curr_count += 1
        if curr_count == n:
            # skip entirely, don't append to out
            curr_count = 0

    counts = Counter(out)

    # remove pairs
    out = []
    for move, count in counts.items():
        inv_move = get_inverse_move(move)
        inv_count = counts.get(inv_move, 0)
        if count > inv_count:
            out.extend([move] * (count - inv_count))
        else:
            out.extend([inv_move] * (inv_count - count))
        if inv_move in counts.keys():
            counts[inv_move] = 0
        counts[move] = 0
    return out


def reduce_cube(subsequence):
    return reduce_commutative(subsequence, n=4)


reduce_subsequence = {
    "cube": reduce_cube,
    "wreath": reduce_wreath,
    "globe": reduce_globe,
}


def iterate_reduction(subsequence, puzzle):
    if puzzle.startswith("globe"):
        n = int(puzzle.split("/")[1])
        reduction_method = partial(reduce_subsequence["globe"], n=n * 2)
    elif puzzle.startswith("cube"):
        reduction_method = reduce_subsequence["cube"]
    else:
        reduction_method = reduce_subsequence["wreath"]

    prev = []
    curr = subsequence

    while prev != curr:
        prev = curr[:]
        curr = reduction_method(curr)

    return curr


# subsequence = ["f0", "f0", "f0", "f0", "f1", "f1", "f1", "-f1", "-f1", "-f0"]
subsequence = ["r0", "r0", "r0", "r0", "r1", "r1", "r1", "-r1", "-r1", "-r0"]
# subsequence = ["f0"]
print(iterate_reduction(subsequence, "globe_3/4"))

['r0', 'r0', 'r0', 'r1']


In [31]:
def reduce_sequence(sequence, puzzle):
    out = []
    curr_face = ""
    curr_subsequence = []
    for move in sequence + ["x"]:
        face = remove_prefix(move, "-")[0]
        if face != curr_face:
            out.extend(iterate_reduction(curr_subsequence, puzzle))
            curr_subsequence = []
        curr_face = face
        curr_subsequence.append(move)

    return out


def iterate_reduce_sequence(sequence, puzzle):
    # return reduce_sequence(sequence)
    prev = []
    curr = sequence
    while prev != curr:
        prev = curr[:]
        curr = reduce_sequence(curr, puzzle)
    return curr


# iterate_reduce_sequence(["f0", "f0", "f0", "f0", "f1", "f1", "f1", "-f1", "-f1", "-f0"])

old_total = 0
new_total = 0

out = []
for puzzle, sequence in tqdm(puzzle_solutions.items()):
    puzzle_name = puzzle[0]
    idx = puzzle[1]

    old_total += len(sequence)
    reduced_sequence = iterate_reduce_sequence(sequence, puzzle_name)
    new_total += len(reduced_sequence)

    out.append((".".join(reduced_sequence)))

print(old_total)
print(new_total)

print(f"reduced {old_total-new_total} moves")

100%|██████████| 398/398 [00:04<00:00, 81.58it/s] 

1220590
1201946
reduced 18644 moves


In [32]:
out_df = pd.DataFrame(out)
out_df.to_csv('./solutions/greedy_all.csv')